### Desafio Seazone

#### Feito por Camila Cristina Teixeira 

Olá,

Neste arquivo eu apresento minha solução para o desafio Seazone para a vaga Analista de dados Júnior.

Primeiramente realizou-se a importação das bibliotecas necessarias:

In [ ]:
import pandas as pd
import seaborn as srn
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest
from sklearn.preprocessing import LabelEncoder

**Observação:** É importante salientar que caso não haja problemas de compatibilidade com a biblioteca missingpy com sua maquina esta parte do codigo apresentada acima pode ser ignorada:

```
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
```
    
Se a instalação dos pacotes foi realizada conforme o Readme nenhuma alteração será necessaria.
<p>

A seguir carregou-se os conjuntos de dados e visualizou-se suas as primeiras linhas.

Os dados se encontram na pasta dados seazone.

In [ ]:
#Carregando os dados

#altere o diretorio para pasta que contem os dados em sua maquina
%cd "C:\Users\camil\Documents\seazone" 


df_detalhes = pd.read_csv('desafio_details.csv')
df_price = pd.read_csv('desafio_priceav.csv')
df_detalhes.head()

In [ ]:
df_price.head()

Observou-se que no dataframe há colunas desnecessarias utilizadas como indice que podem ser removidas sem nenhum prejuizo, conforme apresentado abaixo:

In [ ]:
df_price= df_price.drop("Unnamed: 0", axis =1)
df_price= df_price.drop("Unnamed: 0.1", axis =1)
df_detalhes= df_detalhes.drop("Unnamed: 0", axis =1)

df_price.head()

In [ ]:
df_detalhes.head()

Visualizou-se a dimensão dos dois conjuntos de dados e removemos as linhas duplicadas. Após as alterações observamos novamente a dimensão dos dados e podemos observar que o dataframe "df_price" possuia linhas duplicadas.

Em seguida observamos a quantidade de valores nulos.

In [ ]:
#observado a dimensao dos dataframes
print(f'Dimensão dos dados de preço dos listing')
print(df_detalhes.shape, '\n')
print(f'Dimensão dos dados de caracteristicas dos listing')
print(df_price.shape, '\n')
#removendo linhas duplicadas
df_detalhes = df_detalhes.drop_duplicates()
df_price = df_price.drop_duplicates()
#dimensões após alterações
print(f'Dimensão dos dados de preço dos listing após a remoção de linhas duplicadas')
print(df_detalhes.shape, '\n')
print(f'Dimensão dos dados de caracteristicas dos listing após a remoção de linhas duplicadas')
print(df_price.shape, '\n')
print(f'Valores nulos dos dados de caracteristicas dos listing')
#Valores nulos para os datalhes
print(df_detalhes.isnull().sum(), "\n")
#Valores nulos para os dados de preço
print(f'Valores nulos dos dados de preço dos listing')
print(df_price.isnull().sum())


**Observação:** 
As duas proximas celulas de codigo são de imputação de valores faltantes, caso não deseje realiza-lo pule essas celulas.

Na celula abaixo substituiu-se os valores faltantes pela mediana dos mesmos, como a quantidade de valores nulos é baixa nas colunas "number_of_bedrooms", "number_of_bathrooms" e "numero number_of_reviews" essa aproximação não trará problemas.

In [ ]:
df_detalhes['number_of_bedrooms'] = df_detalhes['number_of_bedrooms'].fillna(df_detalhes['number_of_bedrooms'].median())
df_detalhes['number_of_bathrooms'] = df_detalhes['number_of_bathrooms'].fillna(df_detalhes['number_of_bathrooms'].median())
df_detalhes['number_of_reviews'] = df_detalhes['number_of_reviews'].fillna(df_detalhes['number_of_reviews'].median())


Como a coluna "star_rating" é de importancia no modelo não pode ser excluida apesar de cerca de 40% de seus valores serem nulos, para uma imputação mais acertiva e com menor vies foi utilizado o algortimo de RandomForest para imputar os valores nulos dessa variavel.

In [ ]:
#removendo colunas que não auxiliam na predição
df_detalhes2 = df_detalhes
df_detalhes2 = df_detalhes2.drop(['ad_name','airbnb_listing_id'], axis=1)

#codificando colunas categoricas
labelencoder = LabelEncoder()
df_detalhes2['suburb'] = labelencoder.fit_transform(df_detalhes2['suburb'])
df_detalhes2['is_superhost'] = labelencoder.fit_transform(df_detalhes2['is_superhost'])

#convertendo a coluna 'star_rating' para str para que o modelo não faça uma regressão e sim use o classificador
df_detalhes2['star_rating'] = df_detalhes2['star_rating'].astype(str)

#imputando valores faltantes
imputer = MissForest()
X_imputed = imputer.fit_transform(df_detalhes2)
df_detalhes["star_rating"]= X_imputed

Verificou-se então que não restam mais valores nulos nos dados de caracteristicas dos listings:

In [ ]:
df_detalhes.isnull().sum()

Com o tratamento dos dados ja concluido, realizou-se o primeiro item do desafio:

"Ordene os bairros em ordem crescente de número de listings"

Para isso através da função groupby observou-se a relação dos bairros e números de anúncios conforme apresentado.

In [ ]:
bairro_listining = df_detalhes.groupby(["suburb"]).size()
print(bairro_listining)
bairro_listining.plot.bar(color = 'gray', xlabel = "Bairro", ylabel = "Número de Listings")

Pode-se observar que a ordem crescente de número de listing por bairros é: 

Centro < Lagoa da Conceição < Jurerê < Canasvieiras < Ingleses



O segundo do desafio é:
" Ordene os bairros em ordem crescente de faturamento médio dos listings "

Primeiramente nos dados de preço extraimos através da função groupby o id do listing, a ocupação, pois só ocorre faturamento se o imovel tenha sido alugado e a média de preço do anuncio.

In [ ]:
df1 = df_price.groupby(['airbnb_listing_id', 'occupied'])['price_string'].mean()
df1 = pd.DataFrame(df1.reset_index(name = "faturamento_medio"))
df1.head()

Nos dados de caracteristicas dos listing relacionamos o id do listing com o bairro

In [ ]:
df2 = df_detalhes.groupby('airbnb_listing_id')['suburb'].sum()
df2 = pd.DataFrame(df2.reset_index(name = "bairro"))
df2.head()

In [ ]:
df_fat = pd.merge(df1, df2, how = 'outer', on = 'airbnb_listing_id')
print(df_fat.shape)
df_fat.head()

E finalmente unimos as relações através do numero de id do listing e consegumos encontrar faturamento médio dos listings de cada bairro.

In [ ]:
bairro_faturamento = df_fat.groupby(['bairro', 'occupied'])['faturamento_medio'].mean()
bairro_faturamento = pd.DataFrame(bairro_faturamento.reset_index(name = "faturamento_medio"))
bairro_faturamento = bairro_faturamento[bairro_faturamento["occupied"] == 1 ]
print(bairro_faturamento)
bairro_faturamento.plot.bar(x = "bairro", y = "faturamento_medio", color = 'gray', legend = False, ylabel = "Faturamento Médio", xlabel = "Bairro")

Pode-se observar então que a ordem crescente dos bairros por faturamento é:

Centro < Lagoa da Conceição < Canasvieiras < Ingleses < Jurerê

O terceiro item do desafio é:

"Existem correlações entre as características de um anúncio e seu faturamento? Quais? Explique"

Primeiramente selecionou-se apenas as linhas dos dados em que houve a ocupação e consequentemente faturamento.

In [ ]:
df_price= df_price[df_price['occupied'] == 1]

df_price.head()


Agrupou-se o id do listing pelo seu faturamento

In [ ]:
faturamento = df_price.groupby('airbnb_listing_id')['price_string'].sum()

faturamento = pd.DataFrame(faturamento.reset_index(name = "faturamento"))
faturamento.head()

Uniu-se as informações do conjuto de dados de carcteristicas dos listings (df_detalhes) com o faturamento através do id do anuncio.

O faturamento dos id dos listings que não arrecadaram foram preenchidos por 0 para não constar como valores nulos.

In [ ]:
df_faturamento = pd.merge(df_detalhes, faturamento, how = 'outer', on = 'airbnb_listing_id')
print(df_faturamento.shape)

#Preenchendo valores nulos de Id que não faturaram por 0
df_faturamento['faturamento'] = df_faturamento['faturamento'].fillna(0)

#visualizando
df_faturamento.head()

Calculou-se a correlação.

Primeiramente foi verificada a correlação de Pearson:

In [ ]:
#correlação de pearson
correlation = df_faturamento.corr()
print(correlation['faturamento'])

In [ ]:
plot = srn.heatmap(correlation, annot = True, fmt=".1f", linewidths=.8)
plot

Contatou-se que há correlações positivas bem fracas entre o faturamento e o número de banheiros, número de quartos e número de comentários (correlação de Pearson de aproximadamente 0.1) essas correlações significam que cerca 1% da variação do faturamento pode ser explicada por essas variáveis, sendo ainda que o impacto causado é positivo no sentido de aumentar o faturamento. As variáveis nota do anúncio e superhost tem coeficiente inferior a 0.1, sendo então, considerados desprezíveis pela maioria dos autores. É importante salientar que alguns autores sugerem que as correlações fracas são acima de 0.3 e não de 0.1. Assim, o julgamento deve ser realizado pelo analista que pode calcular um valor de p para auxilia-lo na decisão a um nível de significância e verificar se o coeficiente é significativo ou não. 

O coeficiente de correlação de Pearson mede a correlação linear entre as variáveis, a fim de se verificar se há uma correlação não linear também calculou-se o coeficiente de correlação de Spearman 

In [ ]:
#correlação de sperman
correlation = df_faturamento.corr(method = 'spearman')
print(correlation['faturamento'])

In [ ]:
plot = srn.heatmap(correlation, annot = True, fmt=".1f", linewidths=.8)
plot

Conforme observado os valores não diferem muito do coeficiente de Pearson, possuindo correlações desprezíveis ou bem fracas.

Dessa forma, deve-se buscar outras variáveis que possam ter maior correlação com o faturamento afim de descobrir possíveis formas de maximiza-lo. As variáveis categóricas não dicotômicas bairros e título não são passiveis de aplicação de correlação, porém elas podem influenciar o faturamento, outra forma de associação deve ser utilizada, uma das formas mais comuns é aplicar um teste de associação chamado de teste do qui-quadrado que poderia ser utilizado para avaliar a influência dos barros por exemplo.

O ultimo item do desafio é: 
 "Qual a antecedência média das reservas?
a. Esse número é maior ou menor para finais de semana?"

Primeiramente converteu-se as colunas 'booked_on' e 'date' para o formato datatime:

In [ ]:
df_price['booked_on'] = pd.to_datetime(df_price['booked_on'])
df_price['date'] = pd.to_datetime(df_price['date'])

In [ ]:
df_price.info()

Criou-se a coluna antecendencia que mede a quantidade de dias anteior a data de ocupação que o imovel foi alugado.

In [ ]:
df_price['antecedencia']= abs((df_price['booked_on'] - df_price['date']).dt.days)
df_price.head()

Calculamos o valor médio da antecedencia conforme apresentado abaixo, podemos visualizar que a antecedencia média de foi 37.95 aproximadamente.

In [ ]:
#visualizando a média
print(f'A antecedência média é:{round(df_price["antecedencia"].mean(),2)}')

Observou-se também a antecedênca média para alugar datas para o fim de semana, para isso, criou-se a coluna "dia_semana" para computar o dia da semana de "date" e assim selecionou-se os dias correspondetes a fins de semana e calculou-se a média de antecedencia.

In [ ]:
#criando coluna dia_semana
df_price['dia_semana'] = df_price['date'].dt.strftime('%A')
df_price.head()

In [ ]:
#visualizando os dias da semana e a quantidade de ocupações em cada um deles
df_price.groupby('dia_semana').size()


In [ ]:
#selecionando os finais de semana
df_price = df_price[df_price['dia_semana'].str.contains("S")]
df_price.head()

In [ ]:
#visualizando os dias da semana que foram selecionados e a quantidade de ocupações em cada um deles
df_price.groupby('dia_semana').size()

In [ ]:
#calculando média de antecedência

print(f'A antecedência média para finais de semana é:{round(df_price["antecedencia"].mean(),2)}')

Assim, observou-se que a média de antecedencia para o fim de semana é menor cerca de 37.41, mas a diferença é tão pequena que pode ser considerada desprezivel.